In [1]:
import pandas as pd

df = pd.read_csv("hf://datasets/Shengtao/recipe/recipe.csv",usecols=['title','category','rating','rating_count','ingredients','directions','total_time','calories', 'carbohydrates_g', 'sugars_g', 'fat_g', 'saturated_fat_g','cholesterol_mg', 'protein_g', 'dietary_fiber_g', 'sodium_mg','calcium_mg', 'iron_mg','instructions_list','image'])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [2]:
df.isnull().sum()

,0
title,0
category,0
rating,0
rating_count,0
ingredients,0
directions,0
total_time,1832
calories,99
carbohydrates_g,130
sugars_g,601


In [3]:
df['total_time']=df['total_time'].fillna(df['total_time'].mode().values[0])
df['calories']=df['calories'].fillna(df['calories'].median())
df['carbohydrates_g']=df['carbohydrates_g'].fillna(df['carbohydrates_g'].median())
df['sugars_g']=df['sugars_g'].fillna(df['sugars_g'].median())
df['fat_g']=df['fat_g'].fillna(df['fat_g'].median())
df['saturated_fat_g']=df['saturated_fat_g'].fillna(df['saturated_fat_g'].median())
df['cholesterol_mg']=df['cholesterol_mg'].fillna(df['cholesterol_mg'].median())
df['protein_g']=df['protein_g'].fillna(df['protein_g'].median())
df['dietary_fiber_g']=df['dietary_fiber_g'].fillna(df['dietary_fiber_g'].median())
df['sodium_mg']=df['sodium_mg'].fillna(df['sodium_mg'].median())
df['calcium_mg']=df['calcium_mg'].fillna(df['calcium_mg'].median())
df['iron_mg']=df['iron_mg'].fillna(df['iron_mg'].median())
df['instructions_list']=df['instructions_list'].fillna("Not Available")
df['image']=df['image'].fillna("Not Available")

In [4]:
df.isnull().sum()

,0
title,0
category,0
rating,0
rating_count,0
ingredients,0
directions,0
total_time,0
calories,0
carbohydrates_g,0
sugars_g,0


In [24]:
import re

def convert_to_minutes(time_str):
    days = re.search(r'(\d+)\s*day', time_str)
    hours = re.search(r'(\d+)\s*hr', time_str)
    minutes = re.search(r'(\d+)\s*min', time_str)

    total_minutes = 0
    if days:
        total_minutes += int(days.group(1)) * 24 * 60
    if hours:
        total_minutes += int(hours.group(1)) * 60
    if minutes:
        total_minutes += int(minutes.group(1))

    return total_minutes

# Apply the function
df["total_time"] = df["total_time"].apply(convert_to_minutes)

In [25]:
df['total_time'].sample(10)

,total_time
29466,25
14139,100
3791,40
1301,30
12591,55
14564,35
9327,20
27635,90
7421,65
7563,85


In [5]:
import re

def convert_fraction(fraction_str):
    """
    Convert unicode fractions and regular fractions to float.
    """
    # Unicode fraction mapping
    fraction_map = {
        '¼': 0.25,
        '½': 0.5,
        '¾': 0.75,
        '⅐': 1/7,
        '⅑': 1/9,
        '⅒': 1/10,
        '⅓': 1/3,
        '⅔': 2/3,
        '⅕': 1/5,
        '⅖': 2/5,
        '⅗': 3/5,
        '⅘': 4/5,
        '⅙': 1/6,
        '⅚': 5/6,
        '⅛': 1/8,
        '⅜': 3/8,
        '⅝': 5/8,
        '⅞': 7/8
    }

    # Check if it's a unicode fraction
    if fraction_str in fraction_map:
        return fraction_map[fraction_str]

    # Check if it's a regular fraction (e.g., "1/2")
    if '/' in fraction_str:
        try:
            num, denom = fraction_str.split('/')
            return float(num) / float(denom)
        except ValueError:
            return None

    # Try to convert to float directly
    try:
        return float(fraction_str)
    except ValueError:
        return None

def parse_ingredients(ingredient_string):
    """
    Parse ingredient strings into structured data.

    Parameters:
    ingredient_string (str): Raw ingredient string with quantities and items

    Returns:
    list: List of dictionaries containing parsed ingredient information
    """
    # Split ingredients by semicolon or newline
    ingredients_list = [i.strip() for i in re.split('[;\n]', ingredient_string) if i.strip()]

    parsed_ingredients = []

    for ingredient in ingredients_list:
        # Regular expression to match common quantity patterns including unicode fractions
        quantity_pattern = r'^([\d./¼½¾⅐⅑⅒⅓⅔⅕⅖⅗⅘⅙⅚⅛⅜⅝⅞]+)?\s*(?:\(.*?\))?\s*([\d./¼½¾⅐⅑⅒⅓⅔⅕⅖⅗⅘⅙⅚⅛⅜⅝⅞]+)?\s*(cup|cups|tablespoon|teaspoon|ounce|pound|oz|lb|tbsp|tsp|g|ml|box|piece|to taste)?s?\b'

        match = re.match(quantity_pattern, ingredient, re.IGNORECASE)


        if match:

            # Get the ingredient name by removing the quantity and unit part
            ingredient_name = ingredient[match.end():].strip()
            # Remove 'to taste' from ingredient name if it's not the unit
            ingredient_name = re.sub(r'\s+to\s+taste$', '', ingredient_name)

        else:
            # If no quantity/unit pattern is found, treat entire string as ingredient name
            ingredient_name= ingredient.strip()

        parsed_ingredients.append(ingredient_name)

    return parsed_ingredients

df['parsed_ingredients'] = df['ingredients'].apply(parse_ingredients)

In [6]:
df['ingredients'][2655]

'¼ cup salt ; ¼ cup white sugar ; 2 tablespoons packed brown sugar ; 2 tablespoons ground black pepper ; 2 tablespoons ground white pepper ; 2 tablespoons onion powder ; 1 tablespoon garlic powder ; 1 tablespoon chili powder ; 1 tablespoon ground paprika ; 1 tablespoon ground cumin ; 10 pounds baby back pork ribs ; 1 cup apple juice ; ¼ cup packed brown sugar ; ¼ cup barbeque sauce'

In [7]:
df['parsed_ingredients'][2655]

['salt',
 'white sugar',
 'packed brown sugar',
 'ground black pepper',
 'ground white pepper',
 'onion powder',
 'garlic powder',
 'chili powder',
 'ground paprika',
 'ground cumin',
 'baby back pork ribs',
 'apple juice',
 'packed brown sugar',
 'barbeque sauce']

In [8]:
def clean_instructions(instructions):
    """
    Clean recipe instructions by removing escape characters and normalizing quotes

    Parameters:
    instructions (str): Raw instruction text

    Returns:
    str: Cleaned instruction text
    """
    # Remove escape characters and different types of quotes
    cleaned = instructions.replace('\\\'', "'")  # Replace escaped single quotes
    cleaned = cleaned.replace('\\"', '"')      # Replace escaped double quotes
    cleaned = cleaned.replace('"', '')         # Remove double quotes
    cleaned = cleaned.replace('\'', '')        # Remove single quotes
    cleaned = cleaned.replace('[', '')        # Remove single quotes
    cleaned = cleaned.replace(']', '')        # Remove single quotes

    # Fix spacing between sentences (add space after period if missing)
    cleaned = cleaned.replace('.','. ')
    cleaned = cleaned.replace('  ', ' ')      # Remove any double spaces
    cleaned = cleaned.strip()                 # Remove leading/trailing spaces

    return cleaned

# For the dataframe
df['clean_instructions'] = df['instructions_list'].apply(clean_instructions)

In [9]:
df['instructions_list'][0]

'[\'Bring a large pot of lightly salted water to a boil. Cook elbow macaroni in the boiling water, stirring occasionally until cooked through but firm to the bite, 8 minutes.\', "At the same time, melt butter in a saucepan over medium heat. Add flour, salt, and pepper and stir until smooth, about 5 minutes. Pour in milk slowly, while stirring continuously. Continue to cook and stir until mixture is smooth and bubbling, about 5 minutes, making sure the milk doesn\'t burn.", \'Add Cheddar cheese and stir until melted, 2 to 4 minutes.\', \'Drain macaroni and fold into cheese sauce until coated.\']'

In [10]:
df['clean_instructions'][0]

'Bring a large pot of lightly salted water to a boil. Cook elbow macaroni in the boiling water, stirring occasionally until cooked through but firm to the bite, 8 minutes. , At the same time, melt butter in a saucepan over medium heat. Add flour, salt, and pepper and stir until smooth, about 5 minutes. Pour in milk slowly, while stirring continuously. Continue to cook and stir until mixture is smooth and bubbling, about 5 minutes, making sure the milk doesnt burn. , Add Cheddar cheese and stir until melted, 2 to 4 minutes. , Drain macaroni and fold into cheese sauce until coated.'

In [11]:
df['category']=df['category'].apply(lambda x:x.replace("-"," "))

In [12]:
df.sample(2)

,title,category,rating,rating_count,ingredients,directions,total_time,calories,carbohydrates_g,sugars_g,...,cholesterol_mg,protein_g,dietary_fiber_g,sodium_mg,calcium_mg,iron_mg,instructions_list,image,parsed_ingredients,clean_instructions
32078,Fruit Fusion Smoothie,trusted brands recipes and tips,0.00,0,⅓ cup sliced fresh strawberries ; ⅓ cup cherri...,Place all ingredients into blender and blend o...,5 mins,178.3,38.3,21.9,...,54.9,2.0,3.6,136.4,27.6,0.6,Not Available,Not Available,"[sliced fresh strawberries, cherries, pitted, ...",Not Available
9572,Corn Flake-Crusted Halibut,main dish,4.47,19,"2 cups corn flakes cereal, finely crushed ; 2 ...",Preheat the oven to 450 degrees F (230 degrees...,20 mins,241.7,13.9,2.3,...,48.5,25.0,0.4,321.8,23.3,5.2,Not Available,Not Available,"[corn flakes cereal, finely crushed, chopped f...",Not Available


In [13]:
df = df.drop(columns=['ingredients', 'instructions_list'], axis=1)

In [14]:
df.sample(2)

,title,category,rating,rating_count,directions,total_time,calories,carbohydrates_g,sugars_g,fat_g,saturated_fat_g,cholesterol_mg,protein_g,dietary_fiber_g,sodium_mg,calcium_mg,iron_mg,image,parsed_ingredients,clean_instructions
16578,Easy Taco Rolls,world cuisine,4.44,80,Preheat an oven to 450 degrees F (230 degrees ...,40 mins,621.3,54.1,5.0,30.2,14.4,97.8,33.0,3.9,1370.0,421.4,5.0,https://www.allrecipes.com/thmb/SZQY9BMKQ4TWWU...,"[uncooked instant rice, water, ground beef, pa...",Preheat an oven to 450 degrees F (230 degrees ...
1215,South African Melktert (Milk Tart),world cuisine,4.06,48,Preheat the oven to 375 degrees F (190 degrees...,1 hr 10 mins,294.3,46.8,30.9,8.5,4.9,98.0,7.9,0.3,238.5,189.6,1.6,https://www.allrecipes.com/thmb/B345i6FXvWY8fb...,"[butter, melted, white sugar, egg yolks, cake ...",Preheat the oven to 375 degrees F (190 degrees...


In [15]:
df['clean_instructions'][0]

'Bring a large pot of lightly salted water to a boil. Cook elbow macaroni in the boiling water, stirring occasionally until cooked through but firm to the bite, 8 minutes. , At the same time, melt butter in a saucepan over medium heat. Add flour, salt, and pepper and stir until smooth, about 5 minutes. Pour in milk slowly, while stirring continuously. Continue to cook and stir until mixture is smooth and bubbling, about 5 minutes, making sure the milk doesnt burn. , Add Cheddar cheese and stir until melted, 2 to 4 minutes. , Drain macaroni and fold into cheese sauce until coated.'

In [16]:
df.sample(5)

,title,category,rating,rating_count,directions,total_time,calories,carbohydrates_g,sugars_g,fat_g,saturated_fat_g,cholesterol_mg,protein_g,dietary_fiber_g,sodium_mg,calcium_mg,iron_mg,image,parsed_ingredients,clean_instructions
23731,Mad Hatter Salad,salad,4.74,232,"In a large bowl, toss together the coleslaw mi...",35 mins,395.2,28.9,10.2,28.8,7.3,23.3,8.0,3.7,251.5,77.7,2.0,https://www.allrecipes.com/thmb/TXV2Ik7kt2OKiu...,"[package coleslaw mix, green onions, chopped, ...","Toss together coleslaw mix, broccoli, and gree..."
3036,Leftover Roast Chicken Soup,soups stews and chili,3.40,15,"In a large stock pot place chicken frame, bone...",2 hrs 15 mins,143.8,32.7,2.9,0.2,0.1,0.0,3.9,4.8,22.4,34.1,1.5,https://www.allrecipes.com/thmb/9O-qitPEbAJ2w3...,"[leftover roast chicken frame - bones, giblets...","Place chicken frame, including bones and gible..."
24298,Classic Cheese Fondue,uncategorized,0.00,0,"Add white wine, cherry brandy, garlic clove, c...",25 mins,214.5,2.6,0.9,14.0,8.5,45.6,13.2,1.9,178.8,461.4,0.2,https://www.allrecipes.com/thmb/faVPxUvYG4pySZ...,"[dry white wine (such as Sauvignon Blanc), che...","Add white wine, cherry brandy, garlic clove, c..."
14277,Langues de Chat,world cuisine,4.29,24,Preheat the oven to 400 degrees F (200 degrees...,1 hr 10 mins,83.0,10.2,5.9,4.4,2.7,7.6,1.2,0.5,25.2,1.9,0.5,https://www.allrecipes.com/thmb/XGSwUA1TA68ibI...,"[butter, softened, white sugar, white sugar, e...",Preheat the oven to 400 degrees F (200 degrees...
8770,Homemade Wine,side dish,4.29,132,"Combine the yeast, sugar and juice concentrate...",1 week,120.6,30.8,30.7,0.1,0.0,0.0,0.2,0.1,4.1,5.2,0.1,https://www.allrecipes.com/thmb/hHYXPfgayw-JrM...,"[package active dry yeast, sugar, can frozen j...","Combine sugar, juice concentrate, and yeast in..."


In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def create_simple_recommender(df):
    """
    Create a simple recipe recommendation system using TF-IDF
    """
    # Convert ingredients list to strings
    df['ingredients_str'] = df['parsed_ingredients'].apply(lambda x: ' '.join(x) if isinstance(x, list) else ' '.join(eval(x)))

    # Create TF-IDF vectors
    tfidf = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf.fit_transform(df['ingredients_str'])

    def get_recommendations(recipe_idx, n_recommendations=5):
        """Get recipe recommendations based on ingredient similarity"""
        # Get similarity scores
        cosine_scores = cosine_similarity(tfidf_matrix[recipe_idx:recipe_idx+1], tfidf_matrix).flatten()

        # Get indices of top similar recipes (excluding self)
        similar_indices = np.argsort(cosine_scores)[::-1][1:n_recommendations+1]

        # Return recommendations
        recommendations = df.iloc[similar_indices][['title', 'category', 'rating']]
        recommendations['similarity_score'] = cosine_scores[similar_indices]

        return recommendations

    def get_recommendations_for_new_recipe(ingredients_list, n_recommendations=5):
        """Get recommendations for a new recipe"""
        # Convert ingredients list to string
        ingredients_str = ' '.join(ingredients_list)

        # Transform new recipe ingredients
        new_recipe_vector = tfidf.transform([ingredients_str])

        # Get similarity scores
        cosine_scores = cosine_similarity(new_recipe_vector, tfidf_matrix).flatten()

        # Get indices of top similar recipes
        similar_indices = np.argsort(cosine_scores)[::-1][:n_recommendations]

        # Return recommendations
        recommendations = df.iloc[similar_indices][['title', 'category', 'rating']]
        recommendations['similarity_score'] = cosine_scores[similar_indices]

        return recommendations

    return get_recommendations, get_recommendations_for_new_recipe, tfidf

In [18]:
# Create the recommender system
get_recommendations, get_recommendations_for_new, tfidf = create_simple_recommender(df)

# Get recommendations for an existing recipe (e.g., recipe at index 0)
recommendations = get_recommendations(0, n_recommendations=5)
print("Recommendations for:", df['title'].iloc[0])
print(recommendations)

# Get recommendations for a new recipe
new_recipe_ingredients = ['chicken', 'garlic', 'olive oil', 'lemon']
new_recommendations = get_recommendations_for_new(new_recipe_ingredients, n_recommendations=5)
print("\nRecommendations for new recipe:")
print(new_recommendations)

Recommendations for: Simple Macaroni and Cheese
                                   title          category  rating  \
17394               Bev's Mac and Cheese         main dish    4.20   
29992                 Macaroni Au Gratin  everyday cooking    4.38   
5451                        Macaroni Pie         main dish    3.56   
17610                Mac and Cheese Bake         main dish    4.28   
23551  Grandma's Southern Mac and Cheese         side dish    4.25   

       similarity_score  
17394          0.969404  
29992          0.868808  
5451           0.859323  
17610          0.756786  
23551          0.746650  

Recommendations for new recipe:
                                  title               category  rating  \
24147         Easy Lemon Garlic Chicken       meat and poultry    4.21   
4391   Chicken with 40 Cloves of Garlic       meat and poultry    4.58   
19101       Lemon-Thyme Chicken Breasts       meat and poultry    5.00   
17420  Marinated Rosemary Lemon Chicken       

In [22]:
df.to_csv('transformed_df.csv', encoding = 'utf-8-sig',index=False)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>